In [1]:
import numpy as np
import pandas as pd
import pysubgroup as ps
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
df = pd.read_csv('./Data/tournaments.csv')

#### Tratando dataframe

In [6]:
df.replace(np.nan, 0, inplace=True)
df.tail()

,id_card,name_card,amount_card,price_card,energy_type_card,type_card,combo_type_id,combo_type_name,id_player,name_player,...,region_tournament,country_tournament,year_tournament,month_tournament,day_tournament,valid_rotation_at_tournament,rotation_name,year_begin,month_begin,day_begin
114286,GRI130a,Rescue Stretcher,2,0.72,0,Trainer,47.0,Zoroark Greninja,1900,Jin Woo Lee,...,AS-OC,Korea,2018,12,22,0,standard_2019,2018,8,28
114287,GRI119,Aqua Patch,2,0.27,0,Trainer,47.0,Zoroark Greninja,1900,Jin Woo Lee,...,AS-OC,Korea,2018,12,22,0,standard_2019,2018,8,28
114288,FLF092,Pal Pad,1,0.68,0,Trainer,47.0,Zoroark Greninja,1900,Jin Woo Lee,...,AS-OC,Korea,2018,12,22,0,standard_2019,2018,8,28
114289,GRI121,Choice Band,2,0.20,0,Trainer,47.0,Zoroark Greninja,1900,Jin Woo Lee,...,AS-OC,Korea,2018,12,22,0,standard_2019,2018,8,28
114290,GRI120,Brooklet Hill,2,0.32,0,Trainer,47.0,Zoroark Greninja,1900,Jin Woo Lee,...,AS-OC,Korea,2018,12,22,0,standard_2019,2018,8,28


#### Tentaremos encontrar subgrupos frequentes de cartas que aparecem em torneios diferentes de batalha de cartas pokemon no mesmo ano

- Definindo todas as regioes do dataset

In [4]:
df = df[df['region_tournament'] != 0]

# eliminando cartas do tipo treinador
df = df[df['type_card'] != 'Trainer']
regions = df['region_tournament'].unique()

regions

array(['SA', 'EU', 'AS-OC', 'JP'], dtype=object)

- Definindo treinadores com suas cartas para cada regiao e realizando beamsearth para descobrir padroes de cartas utilizadas por jogadores diferentes

In [5]:
# Agrupando cartas de cada jogador no dataset
def agrupa_jogador_cartas(df):
    dict_jogador_cartas = {}
    for _, row in df.iterrows():
        if row['name_player'] not in dict_jogador_cartas:
            dict_jogador_cartas[row['name_player']] = []
        dict_jogador_cartas[row['name_player']].append(row['name_card'])
    return len(dict_jogador_cartas), dict_jogador_cartas

for region in regions:
    print(f"region: {region}")
    df_region = df[(df['region_tournament'] == region) & (df['year_tournament'] == 2019)]
    jogadores = df_region['name_player'].unique()
    cartas = df_region['name_card'].unique()
    quantidade_jogadores, dict_jogador_cartas = agrupa_jogador_cartas(df_region)
    # Cria um dataframe que as colunas sao todas as cartas existentes no dataset
    df_jogadores_cartas = pd.DataFrame(columns=cartas)
    for jogador in jogadores:
        cartas_jogador = dict_jogador_cartas[jogador]
        linha = []
        for carta in cartas:
            if carta in cartas_jogador:
                linha.append(True)
            else:
                linha.append(False)
        df_jogadores_cartas.loc[len(df_jogadores_cartas)] = linha
    # Apriori
    frequent_itemsets = apriori(df_jogadores_cartas, min_support=0.2, use_colnames=True)
    frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
    rules = rules.sort_values(by='lift', ascending=False).drop(['antecedent support', 'consequent support', 'leverage', 'conviction', 'zhangs_metric'], axis=1)
    print(f"Total de jogadores: {quantidade_jogadores}")
    print(frequent_itemsets)
    print(rules)
    print("\n\n")